# Notebook ① – Data Preparation & Index Build

Prerequisites  
```bash
pip install pandas pyarrow llama-index chromadb sentence-transformers
```

In [1]:
import glob, json, os, pandas as pd
from pathlib import Path

from llama_index.core import Document, ServiceContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.embeddings.utils import resolve_embed_model


DATA_DIR = Path('data')
PICKUP_FILES = glob.glob(str(DATA_DIR/'pickup'/'*.csv'))
DELIVERY_FILES = glob.glob(str(DATA_DIR/'delivery'/'*.csv'))
print(f'Found {len(PICKUP_FILES)} pickup + {len(DELIVERY_FILES)} delivery CSVs')

Found 5 pickup + 5 delivery CSVs


In [2]:
def load_csvs(files):
    dfs = [pd.read_csv(f) for f in files]
    return pd.concat(dfs, ignore_index=True)

pickup_df = load_csvs(PICKUP_FILES)
delivery_df = load_csvs(DELIVERY_FILES)
df = pd.concat([pickup_df, delivery_df], ignore_index=True)
print(df.head())

   order_id  region_id       city  courier_id     accept_time  \
0    483671          3  Chongqing        1518  08-14 07:57:00   
1   1746131          3  Chongqing        4706  10-09 07:46:00   
2   2301722          3  Chongqing        4706  10-09 13:57:00   
3   3788723          3  Chongqing        4706  05-19 08:13:00   
4    713435          3  Chongqing        4706  05-22 08:16:00   

  time_window_start time_window_end        lng       lat  aoi_id  ...  \
0    08-14 09:00:00  08-14 11:00:00  106.46877  29.47204     218  ...   
1    10-09 09:00:00  10-09 11:00:00  106.46872  29.47200     218  ...   
2    10-09 13:57:00  10-09 15:57:00  106.46869  29.47191     218  ...   
3    05-19 11:00:00  05-19 13:00:00  106.46878  29.47208     218  ...   
4    05-22 09:00:00  05-22 11:00:00  106.46813  29.47228     218  ...   

   pickup_gps_lng pickup_gps_lat accept_gps_time  accept_gps_lng  \
0             NaN            NaN             NaN             NaN   
1             NaN            NaN  

In [3]:
# Normalise
# Step 1: Clean and restore string timestamps
df['accept_time'] = df['accept_time'].astype(str).str.strip()

# Step 2: Add year and parse as datetime
df['accept_time'] = pd.to_datetime(
    '2023-' + df['accept_time'],
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce',
    utc=True
)

# Step 3: Optional ISO string version
df['accept_time_iso'] = df['accept_time'].astype(str)

# Step 4: Normalize courier_id
df['courier_id'] = df['courier_id'].astype(str).str.zfill(6)

# Step 5: Sort
df = df.sort_values('accept_time').reset_index(drop=True)


In [4]:
print(df['accept_time'].isna().sum())

0


In [5]:
print(df['accept_time'].astype(str).head(10).tolist())

['2023-04-16 12:33:00+00:00', '2023-04-24 08:18:00+00:00', '2023-04-24 13:05:00+00:00', '2023-04-26 08:13:00+00:00', '2023-04-26 08:37:00+00:00', '2023-04-26 08:40:00+00:00', '2023-04-26 08:41:00+00:00', '2023-04-27 08:38:00+00:00', '2023-04-27 08:49:00+00:00', '2023-04-27 10:55:00+00:00']


In [9]:
print("🟢 Parsed rows:", df['accept_time'].notna().sum())
print("🔴 Failed rows:", df['accept_time'].isna().sum())

🟢 Parsed rows: 10650808
🔴 Failed rows: 0


In [7]:
# Chunk into courier-day docs
grouped = df.groupby([df['courier_id'], df['accept_time'].dt.date])
docs = []
for (cid, day), grp in grouped:
    docs.append(
        Document(
            text=grp.to_json(orient='records'),
            metadata={'courier': cid, 'date': str(day)}
        )
    )
print(f'Created {len(docs):,} documents')    

Created 594,603 documents


In [8]:
print(docs[0].metadata)
print(json.loads(docs[0].text)[0])  # preview 1st row in first doc

{'courier': '000000', 'date': '2023-06-20'}
{'order_id': 4054541, 'region_id': 29, 'city': 'Shanghai', 'courier_id': '000000', 'accept_time': 1687279140000, 'time_window_start': None, 'time_window_end': None, 'lng': 121.41333, 'lat': 31.25387, 'aoi_id': 27388, 'aoi_type': 1, 'pickup_time': None, 'pickup_gps_time': None, 'pickup_gps_lng': None, 'pickup_gps_lat': None, 'accept_gps_time': '06-20 16:39:00', 'accept_gps_lng': 121.40005, 'accept_gps_lat': 31.25108, 'ds': 620, 'delivery_time': '06-20 18:39:00', 'delivery_gps_time': '06-20 18:39:00', 'delivery_gps_lng': 121.39989, 'delivery_gps_lat': 31.25093, 'accept_time_iso': '2023-06-20 16:39:00+00:00'}


In [19]:
import json

json_docs = [
    {"text": d.text, "metadata": d.metadata} for d in docs
]
with open("docs_594k.json", "w", encoding="utf-8") as f:
    json.dump(json_docs, f, ensure_ascii=False)

print("✅ Saved documents to docs_594k.json")


✅ Saved documents to docs_594k.json


In [1]:
import json
from llama_index.core import Document

with open("docs_594k.json", "r", encoding="utf-8") as f:
    json_docs = json.load(f)

# Recreate Document objects
docs = [Document(text=entry["text"], metadata=entry["metadata"]) for entry in json_docs]


In [2]:
import torch

if torch.cuda.is_available():
    print("✅ CUDA GPU available:", torch.cuda.get_device_name(0))
else:
    print("❌ GPU not available, using CPU")


✅ CUDA GPU available: NVIDIA GeForce RTX 5060 Ti


In [3]:
# --- Imports & Config ---
import json
import os
import time
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, VectorStoreIndex, Document
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# --- Load saved documents ---
with open("docs_594k.json", "r", encoding="utf-8") as f:
    json_docs = json.load(f)
docs = [Document(text=d["text"], metadata=d["metadata"]) for d in json_docs]

# --- Set up HuggingFace embedding with GPU + large batch ---
embed_model = HuggingFaceEmbedding(
    model_name="intfloat/e5-base-v2",
    embed_batch_size=256,
    device="cuda"  # ✅ explicitly use GPU
)
Settings.embed_model = embed_model
Settings.chunk_size = 1024

# --- Set up Chroma vector store ---
client = chromadb.PersistentClient(path="lade_chroma")
collection = client.get_or_create_collection("lade")
vector_store = ChromaVectorStore(chroma_collection=collection)

# --- Resume tracking ---
batch_size = 10000  # Smaller chunk for test speed; raise to 50_000 later
total_chunks = (len(docs) + batch_size - 1) // batch_size
checkpoint_file = "embedding_progress.json"

# --- Load completed chunks ---
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        completed = set(json.load(f))
else:
    completed = set()



c:\Users\Admin\Documents\Research LaDe\lade_rag_pipeline\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


> **Next**: open `SyntheticDataset_Creator.ipynb` to build the gold Q‑A set.

In [4]:
with tqdm(total=total_chunks, desc="Embedding chunks", leave=True) as pbar:
    pbar.update(len(completed))  # skip completed

    for i in range(0, len(docs), batch_size):
        chunk_id = i // batch_size
        if chunk_id in completed:
            continue

        chunk = docs[i:i + batch_size]

        start = time.time()
        VectorStoreIndex.from_documents(
            chunk,
            vector_store=vector_store,
            show_progress=True      # 👈 inner progress bar
        )
        elapsed = time.time() - start

        # ✅ Save progress
        completed.add(chunk_id)
        with open(checkpoint_file, "w") as f:
            json.dump(sorted(completed), f)

        pbar.update(1)
        print(f"✅ Finished chunk {chunk_id} ({i}–{i+len(chunk)}) in {elapsed/60:.2f} min")


Embedding chunks:  35%|███▌      | 21/60 [20:15<37:36, 57.87s/it]

✅ Finished chunk 20 (200000–210000) in 20.25 min


Embedding chunks:  37%|███▋      | 22/60 [45:05<1:34:25, 149.09s/it]

✅ Finished chunk 21 (210000–220000) in 24.84 min


Embedding chunks:  38%|███▊      | 23/60 [1:07:30<2:33:28, 248.87s/it]

...

✅ Finished chunk 57 (570000–580000) in 23.38 min


Embedding chunks:  98%|█████████▊| 59/60 [13:42:31<24:06, 1446.66s/it]

✅ Finished chunk 58 (580000–590000) in 22.84 min


Embedding chunks: 100%|██████████| 60/60 [13:52:02<00:00, 832.04s/it] 

✅ Finished chunk 59 (590000–594603) in 9.52 min
